In [1]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Optional

In [2]:
class MDP:
    """
    Markov Decision Process implementation with Value Iteration solver.
    """

    def __init__(self, states: List[int], actions: List[int],
                 transitions: Dict, rewards: Dict, gamma: float = 0.9):

        self.states = states
        self.actions = actions
        self.transitions = transitions
        self.rewards = rewards
        self.gamma = gamma
        self.n_states = len(states)
        self.n_actions = len(actions)

    def get_transition_prob(self, s: int, a: int, s_next: int) -> float:
        """Get transition probability P(s'|s,a)"""
        return self.transitions.get((s, a, s_next), 0.0)

    def get_reward(self, s: int, a: int, s_next: int) -> float:
        """Get reward R(s,a,s')"""
        return self.rewards.get((s, a, s_next), 0.0)